In [1]:
import os
import gzip
import errno
import shutil
import pydicom
import numpy as np
import collections

from PIL import Image
import nibabel as nib

import tifffile as tiff
# from tifffile import imsave
import imageio

import matplotlib.pyplot as plt

from skimage.transform import resize
from torch.utils import data
#
import torch
import random
import numpy as np
import os
import torch.nn as nn
import glob
import tifffile as tiff
import torch.nn.functional as F
# ===================================
from NNUtils import CustomDataset_OCT
from NNMetrics import segmentation_scores, f1_score, hd95, preprocessing_accuracy, intersectionAndUnion

/home/moucheng/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/moucheng/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/moucheng/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/moucheng/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
def test(data_1, model, device, class_no, save_location):

    model.eval()

    data_1_testoutputs = []
    data_1_labels = []

    prediction_map_path = save_location + '/' + 'Results_map'
    output_map_path = save_location + '/' + 'Outputs_digits'
    #
    try:
        os.mkdir(prediction_map_path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass
    #
    try:
        os.mkdir(output_map_path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass

    with torch.no_grad():
        #
        f1_1 = 0
        test_iou_1 = 0
        # test_h_dist_1 = 0
        recall_1 = 0
        precision_1 = 0
        mse_1 = 0
        #
#         f1_2 = 0
#         test_iou_2 = 0
#         # test_h_dist_2 = 0
#         recall_2 = 0
#         precision_2 = 0
#         mse_2 = 0
        #
        # ==============================================
        evaluate_index_all_1 = range(0, len(data_1) - 1)
        #
        # ==============================================
#         evaluate_index_all_2 = range(0, len(data_2) - 1)
        #
        for j, (testimg, testlabel, testimgname) in enumerate(data_1):
            # extract a few random indexs every time in a range of the data
            # ========================================================================
            # ========================================================================
            #
            testlabel_original = testlabel
            
#             (c_, h_, w_) = testlabel_original.shape
            #
#             testlabel_original = testlabel_original.reshape(h_, w_, c_)
            #
            print(testlabel_original.shape)
            #
            testimg = torch.from_numpy(testimg).to(device=device, dtype=torch.float32)
            #
            testlabel = torch.from_numpy(testlabel).to(device=device, dtype=torch.float32)
            #
            c, h, w = testimg.size()
            testimg = testimg.expand(1, c, h, w)
            #
            testoutput_original = model(testimg)
            #
            if class_no == 2:
                #
                testoutput = torch.sigmoid(testoutput_original.view(1, h, w))

                testoutput_original = (testoutput > 0.5).float()

                
                #
            else:
                #
                _, testoutput_original = torch.max(testoutput_original, dim=1)
                #
            #
            data_1_testoutputs.append(testoutput_original)
            data_1_labels.append(testlabel_original)
            
            mean_iu_ = intersectionAndUnion(testoutput_original.cpu().detach(), testlabel.cpu().detach(), class_no)

            f1_, recall_, precision_ = f1_score(testlabel.cpu().detach().numpy(), testoutput_original.cpu().detach().numpy(), class_no)

            mse_ = (np.square(testlabel.cpu().detach().numpy() - testoutput_original.cpu().detach().numpy())).mean()

            f1_1 += f1_
            test_iou_1 += mean_iu_
            recall_1 += recall_
            precision_1 += precision_
            mse_1 += mse_
            #
            # Plotting segmentation:
            testoutput_original = np.asarray(testoutput_original.cpu().detach().numpy(), dtype=np.uint8)
            testoutput_original = np.squeeze(testoutput_original, axis=0)
            testoutput_original = np.repeat(testoutput_original[:, :, np.newaxis], 3, axis=2)
            #
            testlabel_original = np.asarray(testlabel_original, dtype=np.uint8)
            testlabel_original = np.squeeze(testlabel_original, axis=0)
            testlabel_original = np.repeat(testlabel_original[:, :, np.newaxis], 3, axis=2)
            #
            if class_no == 2:
                segmentation_map = np.zeros((h, w, 3), dtype=np.uint8)
                #
                segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 255
                segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                #
            else:
                truth_map = np.zeros((h, w, 3), dtype=np.uint8)
                segmentation_map = np.zeros((h, w, 3), dtype=np.uint8)
                if class_no == 4:
                    # multi class for brats 2018
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 255
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 0
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 255
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 0
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 0
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 255
                    #
                elif class_no == 8:
                    # multi class for cityscapes
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 0, testoutput_original[:, :, 1] == 0, testoutput_original[:, :, 2] == 0)] = 0
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 0, testoutput_original[:, :, 1] == 0, testoutput_original[:, :, 2] == 0)] = 0
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 0, testoutput_original[:, :, 1] == 0, testoutput_original[:, :, 2] == 0)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 255
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 1, testoutput_original[:, :, 1] == 1, testoutput_original[:, :, 2] == 1)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 0
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 0
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 2, testoutput_original[:, :, 1] == 2, testoutput_original[:, :, 2] == 2)] = 255
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 255
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 255
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 3, testoutput_original[:, :, 1] == 3, testoutput_original[:, :, 2] == 3)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 4, testoutput_original[:, :, 1] == 4, testoutput_original[:, :, 2] == 4)] = 153
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 4, testoutput_original[:, :, 1] == 4, testoutput_original[:, :, 2] == 4)] = 51
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 4, testoutput_original[:, :, 1] == 4, testoutput_original[:, :, 2] == 4)] = 255
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 5, testoutput_original[:, :, 1] == 5, testoutput_original[:, :, 2] == 5)] = 255
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 5, testoutput_original[:, :, 1] == 5, testoutput_original[:, :, 2] == 5)] = 102
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 5, testoutput_original[:, :, 1] == 5, testoutput_original[:, :, 2] == 5)] = 178
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 6, testoutput_original[:, :, 1] == 6, testoutput_original[:, :, 2] == 6)] = 255
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 6, testoutput_original[:, :, 1] == 6, testoutput_original[:, :, 2] == 6)] = 165
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 6, testoutput_original[:, :, 1] == 6, testoutput_original[:, :, 2] == 6)] = 0
                    #
                    segmentation_map[:, :, 0][np.logical_and(testoutput_original[:, :, 0] == 7, testoutput_original[:, :, 1] == 7, testoutput_original[:, :, 2] == 7)] = 255
                    segmentation_map[:, :, 1][np.logical_and(testoutput_original[:, :, 0] == 7, testoutput_original[:, :, 1] == 7, testoutput_original[:, :, 2] == 7)] = 0
                    segmentation_map[:, :, 2][np.logical_and(testoutput_original[:, :, 0] == 7, testoutput_original[:, :, 1] == 7, testoutput_original[:, :, 2] == 7)] = 0
                    # ===============================
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 0, testlabel_original[:, :, 1] == 0, testlabel_original[:, :, 2] == 0)] = 0
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 0, testlabel_original[:, :, 1] == 0, testlabel_original[:, :, 2] == 0)] = 0
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 0, testlabel_original[:, :, 1] == 0, testlabel_original[:, :, 2] == 0)] = 0
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 1, testlabel_original[:, :, 1] == 1, testlabel_original[:, :, 2] == 1)] = 0
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 1, testlabel_original[:, :, 1] == 1, testlabel_original[:, :, 2] == 1)] = 255
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 1, testlabel_original[:, :, 1] == 1, testlabel_original[:, :, 2] == 1)] = 0
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 2, testlabel_original[:, :, 1] == 2, testlabel_original[:, :, 2] == 2)] = 0
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 2, testlabel_original[:, :, 1] == 2, testlabel_original[:, :, 2] == 2)] = 0
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 2, testlabel_original[:, :, 1] == 2, testlabel_original[:, :, 2] == 2)] = 255
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 3, testlabel_original[:, :, 1] == 3, testlabel_original[:, :, 2] == 3)] = 255
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 3, testlabel_original[:, :, 1] == 3, testlabel_original[:, :, 2] == 3)] = 255
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 3, testlabel_original[:, :, 1] == 3, testlabel_original[:, :, 2] == 3)] = 0
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 4, testlabel_original[:, :, 1] == 4, testlabel_original[:, :, 2] == 4)] = 153
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 4, testlabel_original[:, :, 1] == 4, testlabel_original[:, :, 2] == 4)] = 51
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 4, testlabel_original[:, :, 1] == 4, testlabel_original[:, :, 2] == 4)] = 255
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 5, testlabel_original[:, :, 1] == 5, testlabel_original[:, :, 2] == 5)] = 255
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 5, testlabel_original[:, :, 1] == 5, testlabel_original[:, :, 2] == 5)] = 102
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 5, testlabel_original[:, :, 1] == 5, testlabel_original[:, :, 2] == 5)] = 178
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 6, testlabel_original[:, :, 1] == 6, testlabel_original[:, :, 2] == 6)] = 255
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 6, testlabel_original[:, :, 1] == 6, testlabel_original[:, :, 2] == 6)] = 165
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 6, testlabel_original[:, :, 1] == 6, testlabel_original[:, :, 2] == 6)] = 0
                    #
                    truth_map[:, :, 0][np.logical_and(testlabel_original[:, :, 0] == 7, testlabel_original[:, :, 1] == 7, testlabel_original[:, :, 2] == 7)] = 255
                    truth_map[:, :, 1][np.logical_and(testlabel_original[:, :, 0] == 7, testlabel_original[:, :, 1] == 7, testlabel_original[:, :, 2] == 7)] = 0
                    truth_map[:, :, 2][np.logical_and(testlabel_original[:, :, 0] == 7, testlabel_original[:, :, 1] == 7, testlabel_original[:, :, 2] == 7)] = 0
            
            # 
            prediction_name = 'seg_' + testimgname + '.png'
            full_error_map_name = os.path.join(prediction_map_path, prediction_name)
            imageio.imsave(full_error_map_name, segmentation_map)
            #
            gt_name = 'gt_' + testimgname + '.png'
            full_gt_map_name = os.path.join(prediction_map_path, gt_name)
            imageio.imsave(full_gt_map_name, truth_map)
    #
    digits_raw_name = 'digits.npy'
    full_digits_map_name = os.path.join(output_map_path, digits_raw_name)
    np.save(full_digits_map_name, digits_raw_name)
    #
    label_raw_name = 'labels.npy'
    full_labels_map_name = os.path.join(output_map_path, label_raw_name)
    np.save(full_labels_map_name, label_raw_name)
    # save numerical results:
    result_dictionary = {'Test IoU data 1': str(test_iou_1 / len(evaluate_index_all_1)),
                         'Test f1 data 1': str(f1_1 / len(evaluate_index_all_1)),
                         'Test recall data 1': str(recall_1 / len(evaluate_index_all_1)),
                         'Test Precision data 1': str(precision_1 / len(evaluate_index_all_1)),
                         'Test MSE data 1': str(mse_1 / len(evaluate_index_all_1))}
    #
    ff_path = prediction_map_path + '/test_result_data.txt'
    ff = open(ff_path, 'w')
    ff.write(str(result_dictionary))
    ff.close()
    #
    return test_iou_1 / len(evaluate_index_all_1), f1_1 / len(evaluate_index_all_1), recall_1 / len(evaluate_index_all_1), precision_1 / len(evaluate_index_all_1), mse_1 / len(evaluate_index_all_1), data_1_testoutputs


In [3]:
data_direcotry = '/home/moucheng/projects_data/OCT/duke_dataset/5/'
# ================================================================
train_image_folder = data_direcotry + 'test_1/' + 'images'
train_label_folder = data_direcotry + 'test_1/' + 'masks'
#
test_dataset = CustomDataset_OCT(train_image_folder, train_label_folder, teacher_student=False, transforms='none')

In [4]:
data_1 = test_dataset
# device = torch.device('cuda:0')
device = torch.device('cpu')
model_path = '/home/moucheng/saved_models_MICCAI_Duke_Results/'
model_name = 'Segnet_Epoch_250_Dataset_duke_Batch_4_Width_64_Loss_ce_Norm_bn_ShuffleTraining_True_Data_Augmentation_Train_all__Data_Augmentation_Test_none__lr_0.001_Repeat_1_Final.pt'
model_full_name = model_path + model_name
model = torch.load(model_full_name, map_location='cpu')
class_no = 8
save_location = '/home/moucheng/visual_results'
iou, f1, recall, precision, mse, data_output = test(data_1, model, device, class_no, save_location)

print('Finished')

/home/moucheng/anaconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.pooling.MaxUnpool2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


(1, 512, 512)


/home/moucheng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(1, 512, 512)
(1, 512, 512)
(1, 512, 512)
(1, 512, 512)
(1, 512, 512)
(1, 512, 512)
(1, 512, 512)
Finished
